
#Imports e Leitura do Dicionáŕio de Dados

In [0]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import numpy as np
import json
import urllib.request
import seaborn as sns

In [0]:
dicionario = pd.read_csv('artifacts/dicionario.csv')
dicionario = dicionario[(dicionario.tabela != 'geolocation') & (dicionario.tabela != 'product_category_name_translation')]
display(dicionario)


# Análise geral - granularidade da Olist

In [0]:
vendas_totais = spark.sql("""
SELECT 
  DATE_TRUNC('month', o.order_purchase_timestamp) AS safra,
  COUNT(DISTINCT oi.order_id) AS total_pedidos,
  SUM(oi.price) AS total_valor
FROM 
  olist.default.order_items oi
JOIN 
  olist.default.orders o
ON 
  oi.order_id = o.order_id
WHERE 
  o.order_status = 'delivered'
GROUP BY 
  DATE_TRUNC('month', o.order_purchase_timestamp)
ORDER BY 
  safra
""").toPandas()

fig, ax1 = plt.subplots(figsize=(14, 6))

color = 'tab:blue'
ax1.set_xlabel('Safra (mês)')
ax1.set_ylabel('Total de pedidos', color=color)
line1, = ax1.plot(vendas_totais['safra'], vendas_totais['total_pedidos'], color=color, marker='o', label='Qtd Pedidos')
ax1.tick_params(axis='y', labelcolor=color)

# Compartilhando eixo x
ax2 = ax1.twinx()  
color = 'tab:green'
ax2.set_ylabel('Valor total vendido (R$)', color=color)
line2, = ax2.plot(vendas_totais['safra'], vendas_totais['total_valor'], color=color, marker='s', linestyle='--', label='Valor Total')
ax2.tick_params(axis='y', labelcolor=color)

plt.title('Total de pedidos e valor vendido por safra (todos os sellers)')
fig.autofmt_xdate(rotation=45)
fig.tight_layout()
plt.grid()

lines = [line1, line2]
labels = [line.get_label() for line in lines]
ax1.legend(lines, labels, loc='upper left')

plt.show()

In [0]:
df = spark.sql("""
    SELECT 
        DATE_TRUNC('month', o.order_purchase_timestamp) AS safra,
        AVG(r.review_score) AS nota_media
    FROM olist.default.orders o
    JOIN olist.default.order_reviews r
    ON o.order_id = r.order_id
    GROUP BY safra
    ORDER BY safra
""").toPandas()

df['safra'] = pd.to_datetime(df['safra'])

plt.figure(figsize=(10, 4))
plt.plot(df['safra'], df['nota_media'], marker='o', color='orange')
plt.title("Nota média de reviews por safra")
plt.ylabel("Nota média")
plt.xlabel("Safra (mês)")
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [0]:
df = spark.sql(""" 
        SELECT 
            DATE_TRUNC('month', o.order_purchase_timestamp) AS safra,
            AVG(oi.price) AS ticket_medio
        FROM olist.default.orders o
        JOIN olist.default.order_items oi ON o.order_id = oi.order_id
        WHERE o.order_status = 'delivered'
        GROUP BY safra
        ORDER BY safra
 """).toPandas()

df['safra'] = pd.to_datetime(df['safra'])

plt.figure(figsize=(10, 4))
plt.plot(df['safra'], df['ticket_medio'], marker='d', color='orange')
plt.title("Ticket médio por safra")
plt.ylabel("R$ ticket médio")
plt.xlabel("Safra (mês)")
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [0]:
fig, ax1 = plt.subplots(figsize=(14, 6))

# Y esquerdo
color1 = 'tab:blue'
ax1.set_xlabel('Safra (mês)')
ax1.set_ylabel('Ticket Médido (R$)', color=color1)
line1, = ax1.plot(df['safra'], df['ticket_medio'], color=color1, marker='o', label='Ticket  Médio')
ax1.tick_params(axis='y', labelcolor=color1)

# Y direito
ax2 = ax1.twinx()
color2 = 'tab:green'
ax2.set_ylabel('Total de Pedidos', color=color2)
line2, = ax2.plot(df['safra'], df['total_pedidos'], color=color2, marker='s', linestyle='--', label='Pedidos')
ax2.tick_params(axis='y', labelcolor=color2)

fig.suptitle('Ticket Médio e Pedidos por safra')
fig.autofmt_xdate()
fig.tight_layout()
ax1.legend(
    [line1, line2], 
    [line.get_label() for line in [line1, line2]], 
    loc='lower right'
)
plt.grid(True)
plt.show()


In [0]:
df = spark.sql(""" 
        SELECT 
            DATE_TRUNC('month', order_purchase_timestamp) AS safra,
            COUNT(*) AS total_entregas,
            SUM(CASE WHEN order_delivered_customer_date > order_estimated_delivery_date THEN 1 ELSE 0 END) AS atrasadas,
            100.0 * SUM(CASE WHEN order_delivered_customer_date > order_estimated_delivery_date THEN 1 ELSE 0 END) / COUNT(*) AS perc_atrasadas
        FROM olist.default.orders
        WHERE order_status != 'canceled' OR order_status != 'unavailable'
        GROUP BY safra
        ORDER BY safra
 """).toPandas()

df['safra'] = pd.to_datetime(df['safra'])

plt.figure(figsize=(10, 4))
plt.plot(df['safra'], df['perc_atrasadas'], marker='s', color='orange')
plt.title("Percentual de entregas atrasadas por safra")
plt.ylabel("% atrasadas")
plt.xlabel("Safra (mês)")
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [0]:
df_tempo_entrega = spark.sql("""
SELECT 
  DATEDIFF(order_delivered_customer_date, order_purchase_timestamp) AS dias_entrega
FROM olist.default.orders
WHERE order_status = 'delivered'
  AND order_delivered_customer_date IS NOT NULL
  AND order_purchase_timestamp IS NOT NULL
""").toPandas()

# Filtrar outliers
df_plot = df_tempo_entrega[df_tempo_entrega['dias_entrega'].between(0, 60)]

plt.figure(figsize=(10, 5))
plt.hist(df_plot['dias_entrega'], bins=30, color='mediumslateblue', edgecolor='black')
plt.title("Distribuição do tempo de entrega (em dias)")
plt.xlabel("Dias entre compra e entrega")
plt.ylabel("Frequência de pedidos")
plt.grid(axis='y')
plt.tight_layout()
plt.show()

In [0]:
df_safra_avaliacoes = spark.sql("""
SELECT 
  DATE_TRUNC('month', o.order_purchase_timestamp) AS safra,
  AVG(r.review_score) AS nota_media,
  100.0 * SUM(CASE WHEN o.order_delivered_customer_date > o.order_estimated_delivery_date THEN 1 ELSE 0 END) 
    / COUNT(*) AS perc_atrasos
FROM olist.default.orders o
JOIN olist.default.order_reviews r ON o.order_id = r.order_id
WHERE o.order_status = 'delivered'
  AND o.order_delivered_customer_date IS NOT NULL
  AND o.order_estimated_delivery_date IS NOT NULL
GROUP BY safra
ORDER BY safra
""").toPandas()

df_safra_avaliacoes['safra'] = pd.to_datetime(df_safra_avaliacoes['safra'])

fig, ax1 = plt.subplots(figsize=(14, 5))

# Eixo Y esquerdo
color = 'tab:blue'
ax1.set_xlabel("Safra (mês)")
ax1.set_ylabel("Nota média", color=color)
ax1.plot(df_safra_avaliacoes["safra"], df_safra_avaliacoes["nota_media"], color=color, marker='o', label='Nota média')
ax1.tick_params(axis='y', labelcolor=color)

# Eixo Y direito
ax2 = ax1.twinx()
color = 'tab:red'
ax2.set_ylabel("% atrasos", color=color)
ax2.plot(df_safra_avaliacoes["safra"], df_safra_avaliacoes["perc_atrasos"], color=color, marker='s', linestyle='--', label='% Atrasos')
ax2.tick_params(axis='y', labelcolor=color)

lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines1 + lines2, labels1 + labels2, loc="upper center")

plt.title("Nota média e Percentual de atrasos por safra")
fig.autofmt_xdate()
plt.grid(True)
plt.tight_layout()
plt.show()

In [0]:
query = '''
select 
  date_trunc('month', o.order_purchase_timestamp) as safra,
  sum(oi.freight_value) as total_freight
from olist.default.orders o
left join olist.default.order_items oi 
  on o.order_id = oi.order_id
where o.order_status = 'delivered' AND oi.freight_value is not null
group by safra
order by safra
'''

df = spark.sql(query).toPandas()

df['safra'] = pd.to_datetime(df['safra'])

plt.figure(figsize=(14, 6))
plt.plot(df['safra'], df['total_freight'], marker='s', color='darkorange')
plt.title('Frete Total por safra')
plt.xlabel('safra (mês)')
plt.ylabel('Valor Total (R$)')
plt.grid(axis='y')
plt.legend()
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [0]:
df = spark.sql(
'''
    select 
        date_trunc('month', o.order_purchase_timestamp) as safra,
        count(distinct oi.order_id) as total_pedidos,
        sum(oi.price) as total_valor_vendido,
        total_valor_vendido/count(distinct oi.order_id) as ticket_medio,
        sum(oi.freight_value) as total_valor_frete
    from olist.default.order_items oi
    join olist.default.orders o 
        on oi.order_id = o.order_id
    where o.order_status = 'delivered'
        and oi.price is not null
    group by safra
    order by safra
'''
).toPandas()

df['safra'] = pd.to_datetime(df['safra'])

fig, ax1 = plt.subplots(figsize=(14, 6))

# Y esquerdo
color1 = 'tab:blue'
ax1.set_xlabel('Safra (mês)')
ax1.set_ylabel('Valor vendido (R$)', color=color1)
line1, = ax1.plot(df['safra'], df['total_valor_vendido'], color=color1, marker='o', label='Total vendido')
ax1.tick_params(axis='y', labelcolor=color1)

# Y direito
ax2 = ax1.twinx()
color2 = 'tab:green'
ax2.set_ylabel('Valor arrecadado de frete (R$)', color=color2)
line2, = ax2.plot(df['safra'], df['total_valor_frete'], color=color2, marker='s', linestyle='--', label='Total frete')
ax2.tick_params(axis='y', labelcolor=color2)

fig.suptitle('Valores arrecadados por safra')
fig.autofmt_xdate()
fig.tight_layout()
ax1.legend(
    [line1, line2], 
    [line.get_label() for line in [line1, line2]], 
    loc='upper left'
)
plt.grid(True)
plt.show()


In [0]:
df = spark.sql('''
 select
    date_trunc('month', o.order_purchase_timestamp) as safra,
    avg(date_diff(o.order_delivered_customer_date, o.order_purchase_timestamp)) as tempo_medio_entrega,
    avg(oi.freight_value) as valor_medio_frete
from olist.default.orders o
inner join olist.default.order_items oi
on o.order_id = oi.order_id
where o.order_status != 'canceled'
    and o.order_status != 'unavailable'
group by safra
order by safra
''').toPandas()

df['safra'] = pd.to_datetime(df['safra'])

fig, ax1 = plt.subplots(figsize=(14, 6))

color = 'tab:blue'
ax1.set_xlabel('safra (mês)')
ax1.set_ylabel('Tempo Médio de Entrega (dias)', color=color)
ax1.plot(df['safra'], df['tempo_medio_entrega'], marker='o', color=color)
ax1.tick_params(axis='y', labelcolor=color)
ax1.grid(axis='y')
ax1.grid(axis='x')

ax2 = ax1.twinx()
color = 'tab:orange'
ax2.set_ylabel('Valor Médio de Frete (R$)', color=color)
ax2.plot(df['safra'], df['valor_medio_frete'], marker='s', color=color)
ax2.tick_params(axis='y', labelcolor=color)
ax2.grid(axis='y')
ax2.grid(axis='x')

fig.tight_layout()
plt.title('Tempo Médio de Entrega e Valor Médio de Frete por Safra')
fig.legend(['Tempo Médio de Entrega', 'Valor Médio de Frete'], loc='center')
plt.xticks(rotation=45)
plt.show()


# Análise por Estado

In [0]:
df_valor_estado = spark.sql("""
SELECT 
  c.customer_state,
  SUM(oi.price) AS total_vendido
FROM olist.default.orders o
JOIN olist.default.customers c ON o.customer_id = c.customer_id
JOIN olist.default.order_items oi ON o.order_id = oi.order_id
WHERE o.order_status = 'delivered'
GROUP BY c.customer_state
ORDER BY total_vendido DESC
""").toPandas()

plt.figure(figsize=(12, 5))
plt.bar(df_valor_estado['customer_state'], df_valor_estado['total_vendido'], color='mediumseagreen')
plt.title("Valor total vendido por estado")
plt.ylabel("R$ total vendido")
plt.xlabel("Estado")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [0]:
df_estado = spark.sql("""
SELECT 
  c.customer_state,
  AVG(r.review_score) AS nota_media,
  AVG(oi.price) AS ticket_medio
FROM olist.default.orders o
JOIN olist.default.customers c ON o.customer_id = c.customer_id
JOIN olist.default.order_reviews r ON o.order_id = r.order_id
JOIN olist.default.order_items oi ON o.order_id = oi.order_id
WHERE o.order_status = 'delivered'
GROUP BY c.customer_state
ORDER BY ticket_medio DESC
""").toPandas()

fig, ax1 = plt.subplots(figsize=(12, 5))

# Y esquerdo
color = 'tab:blue'
ax1.set_xlabel("Estado")
ax1.set_ylabel("Nota média", color=color)
ax1.bar(df_estado["customer_state"], df_estado["nota_media"], color=color, alpha=0.6, label="Nota média")
ax1.tick_params(axis='y', labelcolor=color)

# Y direito
ax2 = ax1.twinx()
color = 'tab:green'
ax2.set_ylabel("Ticket médio (R$)", color=color)
ax2.plot(df_estado["customer_state"], df_estado["ticket_medio"], color=color, marker='o', label="Ticket médio")
ax2.tick_params(axis='y', labelcolor=color)

lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines1 + lines2, labels1 + labels2, loc="upper center")

plt.title("Nota média e Ticket médio por Estado")
plt.tight_layout()
plt.grid(True)
plt.show()

In [0]:
df = spark.sql("""
    SELECT 
        c.customer_state,
        count(*) as total_pedidos,
        sum(CASE WHEN o.order_delivered_customer_date > o.order_estimated_delivery_date THEN 1 ELSE 0 END) as atrasos,
        round(100* atrasos/total_pedidos, 2) as perc_atrasos
    FROM olist.default.orders o
    INNER JOIN olist.default.customers c
    ON o.customer_id = c.customer_id
    WHERE o.order_status = 'delivered'
    AND o.order_delivered_customer_date IS NOT NULL
    AND o.order_estimated_delivery_date IS NOT NULL
    GROUP BY c.customer_state
    HAVING COUNT(*) >= 500
    ORDER BY perc_atrasos DESC
""").toPandas()

plt.Figure(figsize=(14, 10))
sns.barplot(x='customer_state', y='perc_atrasos', data=df)
plt.title('Percentual de Pedidos Atrasados por Estado')
plt.xlabel('Estado')
plt.ylabel('Percentual de Pedidos Atrasados (%)')
plt.xticks(rotation=45)
plt.show()

In [0]:
df = spark.sql('''
select
  c.customer_state as estado,
  avg(oi.freight_value) as frete_medio
from olist.default.customers c
inner join olist.default.orders o 
  on c.customer_id = o.customer_id
inner join olist.default.order_items oi
  on o.order_id = oi.order_id
where o.order_status != 'canceled' 
  and o.order_status != 'unavailable'
group by estado
order by frete_medio desc
limit 10
''').toPandas()

plt.Figure(figsize=(14, 10))
sns.barplot(x='estado', y='frete_medio', data=df)
plt.title('Frete Médio por Estado')
plt.xlabel('Estado')
plt.ylabel('Frete Médio')
plt.xticks(rotation=45)
plt.show()

In [0]:
df = spark.sql('''
with base as (
select
  o.order_id as pedido,
  case when o.order_delivered_customer_date > o.order_estimated_delivery_date then 1 else 0 end as atrasado
from olist.default.orders o
inner join olist.default.customers c
  on o.customer_id = c.customer_id
where c.customer_city = 'sao paulo'
  and o.order_status = 'delivered'
  and o.order_delivered_customer_date is not null
  and o.order_estimated_delivery_date is not null
 )
select
  date_trunc('month', o.order_purchase_timestamp) as safra,
  count(*) as total_pedidos,
  sum(b.atrasado) as pedidos_atrasados,
  round(100*pedidos_atrasados/total_pedidos, 2) as perc_atrasados
from olist.default.orders o 
inner join base b 
  on o.order_id = b.pedido
group by safra
order by safra
''').toPandas()

df['safra'] = pd.to_datetime(df['safra'])

fig, ax1 = plt.subplots(figsize=(14, 6))
plt.title('Percentual de Pedidos Atrasados por Safra -  Cidade Sao Paulo')
plt.plot(df['safra'], df['perc_atrasados'], color='tab:blue', marker='o', label='Percentual de Pedidos Atrasados')
plt.xlabel('Safra (mês)')
plt.ylabel('Percentual de Pedidos Atrasados (%)')
plt.xticks(rotation=45)
plt.grid(True)
plt.show()

In [0]:
df_mapa_estado = spark.sql("""
SELECT 
  c.customer_state,
  COUNT(*) / COUNT(DISTINCT DATE_TRUNC('month', o.order_purchase_timestamp)) AS media_mensal_vendas
FROM olist.default.orders o
JOIN olist.default.customers c ON o.customer_id = c.customer_id
WHERE o.order_status = 'delivered'
GROUP BY c.customer_state
""").toPandas()

# Pegandod geoson do mapa do BR
url = 'https://raw.githubusercontent.com/codeforamerica/click_that_hood/master/public/data/brazil-states.geojson'
with urllib.request.urlopen(url) as response:
    brasil_geo = json.load(response)

# Mapear siglas para nomes compatíveis com o GeoJSON
sigla_to_nome = {
    'AC': 'Acre', 'AL': 'Alagoas', 'AP': 'Amapá', 'AM': 'Amazonas', 'BA': 'Bahia',
    'CE': 'Ceará', 'DF': 'Distrito Federal', 'ES': 'Espírito Santo', 'GO': 'Goiás',
    'MA': 'Maranhão', 'MT': 'Mato Grosso', 'MS': 'Mato Grosso do Sul', 'MG': 'Minas Gerais',
    'PA': 'Pará', 'PB': 'Paraíba', 'PR': 'Paraná', 'PE': 'Pernambuco', 'PI': 'Piauí',
    'RJ': 'Rio de Janeiro', 'RN': 'Rio Grande do Norte', 'RS': 'Rio Grande do Sul',
    'RO': 'Rondônia', 'RR': 'Roraima', 'SC': 'Santa Catarina', 'SP': 'São Paulo',
    'SE': 'Sergipe', 'TO': 'Tocantins'
}

df_mapa_estado['estado_nome'] = df_mapa_estado['customer_state'].map(sigla_to_nome)

# Criar o mapa
fig = px.choropleth(
    df_mapa_estado,
    geojson=brasil_geo,
    featureidkey="properties.name",  # chave do nome do estado no GeoJSON
    locations="estado_nome",
    color="media_mensal_vendas",
    color_continuous_scale="Viridis",
    scope="south america",
    title="Volume médio mensal de vendas por estado (Brasil)",
    labels={"media_mensal_vendas": "Média mensal de vendas"}
)

fig.update_geos(fitbounds="locations", visible=False)
fig.update_layout(margin={"r":0,"t":50,"l":0,"b":0})
fig.show()


# Análise por Sellers

In [0]:
top10_df = spark.sql("""
SELECT seller_id
FROM (
  SELECT 
    seller_id,
    COUNT(DISTINCT order_id) AS total_pedidos
  FROM olist.default.order_items
  GROUP BY seller_id
)
ORDER BY total_pedidos DESC
LIMIT 4
""").toPandas()

top10_ids = top10_df['seller_id'].tolist()

# Pegar as vendas desses sellers
vendas_top10 = spark.sql(f"""
SELECT 
  DATE_TRUNC('month', o.order_purchase_timestamp) AS safra,
  oi.seller_id,
  COUNT(DISTINCT oi.order_id) AS qtd_pedidos,
  SUM(oi.price) AS valor_total
FROM 
  olist.default.order_items oi
JOIN 
  olist.default.orders o
ON 
  oi.order_id = o.order_id
WHERE 
  o.order_status = 'delivered'
  AND seller_id IN ({','.join([f"'{s}'" for s in top10_ids])})
GROUP BY 
  seller_id, DATE_TRUNC('month', o.order_purchase_timestamp)
""").toPandas()

vendas_top10['safra'] = pd.to_datetime(vendas_top10['safra'])

vendas_top10 = vendas_top10.sort_values(['seller_id', 'safra'])

plt.figure(figsize=(14, 6))
for seller in top10_ids:
    data = vendas_top10[vendas_top10['seller_id'] == seller]
    plt.plot(data['safra'], data['qtd_pedidos'], marker='o', label=f'{seller[:6]}...')

plt.title('Quantidade de pedidos por safra - Top Sellers')
plt.xlabel('Safra (mês)')
plt.ylabel('Qtd pedidos')
plt.legend()
plt.xticks(rotation=45)
plt.grid()
plt.tight_layout()
plt.show()

plt.figure(figsize=(14, 6))
for seller in top10_ids:
    data = vendas_top10[vendas_top10['seller_id'] == seller]
    plt.plot(data['safra'], data['valor_total'], marker='s', linestyle='--', label=f'{seller[:6]}...')

plt.title('Valor total vendido por safra (R$) - Top Sellers')
plt.xlabel('Safra (mês)')
plt.ylabel('Valor vendido (R$)')
plt.legend()
plt.xticks(rotation=45)
plt.grid()
plt.tight_layout()
plt.show()


In [0]:
top4_sellers = spark.sql('''
    select seller_id 
    from (
        select
        seller_id,
        count(distinct order_id) as total_pedidos
        from olist.default.order_items
        group by seller_id
        order by total_pedidos desc
        limit 4
    )
'''
).toPandas()

lista_sellers = ",".join([f"'{s}'" for s in top4_sellers['seller_id']])

df = spark.sql(
    f'''
    select 
      date_trunc('month', o.order_purchase_timestamp) as safra,
      oi.seller_id,
      sum(oi.freight_value) as total_freight
    from olist.default.orders o
    left join olist.default.order_items oi
        on o.order_id = oi.order_id
    where o.order_status = 'delivered' 
        and oi.seller_id in ({lista_sellers})
        and oi.freight_value is not null
    group by safra, oi.seller_id
    order by safra
    '''
).toPandas()

df['safra'] = pd.to_datetime(df['safra'])

seller_ids = df.seller_id.unique()
seller_labels = {id: f'seller_{chr(65+i)}' for i, id in enumerate(seller_ids)}
df['seller_label'] = df['seller_id'].map(seller_labels)

plt.figure(figsize=(14, 6))

for seller in df['seller_label'].unique():
    dados = df[df.seller_label == seller]
    plt.plot(dados['safra'], dados['total_freight'], label=seller)

plt.title('Frete total arrecadado por safra (R$) - Top4 Sellers')
plt.xlabel('Safra (mês)')
plt.ylabel('Frete Arrecadado (R$)')
plt.legend()
plt.xticks(rotation=45)
plt.grid()
plt.tight_layout()
plt.show()

In [0]:
df_seller = spark.sql("""
WITH vendas_por_seller AS (
  SELECT 
    oi.seller_id,
    AVG(r.review_score) AS nota_media,
    AVG(oi.price) AS ticket_medio
  FROM olist.default.order_items oi
  JOIN olist.default.orders o ON oi.order_id = o.order_id
  JOIN olist.default.order_reviews r ON o.order_id = r.order_id
  WHERE o.order_status = 'delivered'
  GROUP BY oi.seller_id
),
top4 AS (
  SELECT seller_id
  FROM olist.default.order_items
  GROUP BY seller_id
  ORDER BY COUNT(*) DESC
  LIMIT 4
)
SELECT v.*
FROM vendas_por_seller v
JOIN top4 t ON v.seller_id = t.seller_id
""").toPandas()

df_seller = df_seller.copy()
df_seller['seller_label'] = ['Seller A', 'Seller B', 'Seller C', 'Seller D']

x = np.arange(len(df_seller))
width = 0.35  # Largura das barras

fig, ax = plt.subplots(figsize=(10, 5))

bars1 = ax.bar(x - width/2, df_seller['nota_media'], width, label='Nota média', color='cornflowerblue')
bars2 = ax.bar(x + width/2, df_seller['ticket_medio'], width, label='Ticket médio (R$)', color='mediumseagreen')

ax.set_xlabel('Seller')
ax.set_title('Nota média e Ticket médio por Seller (Top 4)')
ax.set_xticks(x)
ax.set_xticklabels(df_seller['seller_label'])
ax.legend()
ax.grid(axis='y')

for bar in bars1:
    height = bar.get_height()
    ax.annotate(f'{height:.1f}',
                xy=(bar.get_x() + bar.get_width() / 2, height),
                xytext=(0, 3),
                textcoords="offset points",
                ha='center', va='bottom',
                fontsize=9, color='blue')

for bar in bars2:
    height = bar.get_height()
    ax.annotate(f'R${height:.0f}',
                xy=(bar.get_x() + bar.get_width() / 2, height),
                xytext=(0, 3),
                textcoords="offset points",
                ha='center', va='bottom',
                fontsize=9, color='green')
plt.tight_layout()
plt.show()

In [0]:
df_atraso_nota_seller = spark.sql("""
WITH sellers_top10 AS (
  SELECT seller_id
  FROM olist.default.order_items
  GROUP BY seller_id
  ORDER BY COUNT(DISTINCT order_id) DESC
  LIMIT 10
),
base AS (
  SELECT 
    oi.seller_id,
    o.order_id,
    r.review_score,
    CASE 
      WHEN o.order_delivered_customer_date > o.order_estimated_delivery_date THEN 1 
      ELSE 0 
    END AS atraso
  FROM olist.default.order_items oi
  JOIN olist.default.orders o ON oi.order_id = o.order_id
  JOIN olist.default.order_reviews r ON o.order_id = r.order_id
  WHERE o.order_status = 'delivered'
    AND o.order_delivered_customer_date IS NOT NULL
    AND o.order_estimated_delivery_date IS NOT NULL
    AND oi.seller_id IN (
      SELECT seller_id FROM sellers_top10
    )
)
SELECT 
  seller_id,
  AVG(review_score) AS nota_media,
  100.0 * SUM(atraso) / COUNT(*) AS perc_atrasos
FROM base
GROUP BY seller_id
""").toPandas()

# Label mais legíveis pros sellers
df_atraso_nota_seller['seller_label'] = ['Seller ' + chr(65+i) for i in range(len(df_atraso_nota_seller))]

x = np.arange(len(df_atraso_nota_seller))
width = 0.35

fig, ax = plt.subplots(figsize=(12,8))

bars1 = ax.bar(x - width/2, df_atraso_nota_seller['nota_media'], width, label='Nota média', color='cornflowerblue')
bars2 = ax.bar(x + width/2, df_atraso_nota_seller['perc_atrasos'], width, label='% atrasos', color='darkorange')

ax.set_xlabel('Seller')
ax.set_title('Nota média e Percentual de Atrasos (Top 10 Sellers)')
ax.set_xticks(x)
ax.set_xticklabels(df_atraso_nota_seller['seller_label'])
ax.legend()
ax.grid(axis='y')

for bar in bars2:
    height = bar.get_height()
    ax.annotate(f'{height:.1f}%',
                xy=(bar.get_x() + bar.get_width() / 2, height),
                xytext=(0, 3),
                textcoords="offset points",
                ha='center', va='bottom',
                fontsize=9, color='black')

plt.tight_layout()
plt.show()


# Anáĺise Categorias de Prdutos

In [0]:
df_top_categorias = spark.sql("""
SELECT 
  p.product_category_name,
  COUNT(*) AS total_vendas
FROM olist.default.order_items oi
JOIN olist.default.products p ON oi.product_id = p.product_id
JOIN olist.default.orders o ON oi.order_id = o.order_id
WHERE o.order_status = 'delivered'
GROUP BY p.product_category_name
ORDER BY total_vendas DESC
LIMIT 10
""").toPandas()

plt.figure(figsize=(12, 5))
sns.barplot(x='product_category_name', y='total_vendas', data=df_top_categorias)
plt.title("Top 10 categorias de produto mais vendidas")
plt.ylabel("Total de vendas")
plt.xlabel("Categoria")
plt.xticks(rotation=45)
plt.grid(axis='y')
plt.tight_layout()
plt.show()


In [0]:
top10_categorias = df_top_categorias['product_category_name'].tolist()

# Convertendo para string SQL
categorias_sql = ','.join([f"'{c}'" for c in top10_categorias])

df_categorias_safra = spark.sql(f"""
SELECT 
  DATE_TRUNC('month', o.order_purchase_timestamp) AS safra,
  p.product_category_name,
  COUNT(*) AS qtd_vendas
FROM olist.default.order_items oi
JOIN olist.default.products p ON oi.product_id = p.product_id
JOIN olist.default.orders o ON oi.order_id = o.order_id
WHERE o.order_status = 'delivered'
  AND p.product_category_name IN ({categorias_sql})
GROUP BY safra, p.product_category_name
ORDER BY safra
""").toPandas()

df_categorias_safra['safra'] = pd.to_datetime(df_categorias_safra['safra'])

plt.figure(figsize=(14, 6))
for categoria in top10_categorias:
    dados = df_categorias_safra[df_categorias_safra['product_category_name'] == categoria]
    plt.plot(dados['safra'], dados['qtd_vendas'], label=categoria)

plt.title('Curva de vendas por safra - Top 10 categorias')
plt.xlabel('Safra (mês)')
plt.ylabel('Quantidade de vendas')
plt.legend()
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [0]:
df = spark.sql('''
with info as (
select
    oi.product_id as produto,
    orv.review_score as nota
from olist.default.orders o
inner join olist.default.order_items oi 
    on o.order_id = oi.order_id
inner join olist.default.order_reviews orv 
    on o.order_id = orv.order_id
where o.order_status != 'canceled' 
    and o.order_status != 'unavailable'
)
select
    p.product_category_name as categoria,
    avg(nota) as nota_media
from olist.default.products p
inner join info i
    on p.product_id = i.produto
group by categoria
order by nota_media desc
limit 10
''').toPandas()

plt.figure(figsize=(14, 6))
sns.barplot(x='categoria', y='nota_media', data=df)
plt.title('Top10 Categorias com Maior Nota Média')
plt.xticks(rotation=75)
plt.xlabel('Categoria')
plt.ylabel('Nota Média')
plt.show()

In [0]:
df = spark.sql('''
    select
        p.product_category_name as categoria,
        count(o.order_id) as qtd_pedidos,
        sum(case when (o.order_delivered_customer_date > o.order_estimated_delivery_date) then 1 else 0 end) as total_atrasados,
        round(100*total_atrasados/qtd_pedidos, 2) as perc_atrasados
    from olist.default.order_items oi
    inner join olist.default.orders o
        on oi.order_id = o.order_id
    inner join olist.default.products p
        on oi.product_id = p.product_id
    where o.order_status = 'delivered'
        and o.order_delivered_customer_date is not null
        and o.order_estimated_delivery_date is not null
        and p.product_category_name is not null
    group by p.product_category_name
    order by perc_atrasados desc
    limit 10               
''').toPandas()

plt.figure(figsize=(14, 6))
sns.barplot(x='categoria', y='perc_atrasados', data=df)
plt.title('Top10 Categorias com Maior Percentual de Pedidos Atrasados')
plt.xlabel('Categoria')
plt.ylabel('Percentual de Pedidos Atrasados (%)')
plt.xticks(rotation=75)
plt.show()

In [0]:
df = spark.sql('''
select
  p.product_category_name as categoria,
  avg(oi.freight_value) as media_frete
from olist.default.order_items oi
inner join olist.default.products p
  on p.product_id = oi.product_id
group by p.product_category_name
order by media_frete desc
limit 10
''').toPandas()

plt.figure(figsize=(14, 6))
sns.barplot(x='categoria', y='media_frete', data=df)
plt.title('Top10 Categorias Fretes Mais Caros')
plt.xlabel('Categoria')
plt.ylabel('Média de Frete')
plt.xticks(rotation=75)
plt.show()

In [0]:
df = spark.sql('''
select
  p.product_category_name as produto,
  avg(oi.price) as preco_medio
from olist.default.orders o
inner join olist.default.order_items oi
  on o.order_id = oi.order_id
inner join olist.default.products p
  on oi.product_id = p.product_id
where o.order_status != 'canceled'
  and o.order_status != 'unavailable'
group by p.product_category_name
order by preco_medio desc
limit 10
''').toPandas()

plt.figure(figsize=(14, 6))
sns.barplot(x='produto', y='preco_medio', data=df)
plt.title('Top10 Categorias mais Caras')
plt.xlabel('Categoria')
plt.ylabel('Preço Médio (R$)')
plt.xticks(rotation=75)
plt.show()

In [0]:
df = spark.sql('''
with vendas as (
select
    date_trunc('month', o.order_purchase_timestamp) as safra,
    o.order_id as pedido,
    oi.product_id as produto,
    case when o.order_delivered_customer_date > o.order_estimated_delivery_date then 1 else 0 end as atrasado
from olist.default.orders o
inner join olist.default.order_items oi
    on o.order_id = oi.order_id
where o.order_status != 'canceled'
    and o.order_status != 'unavailable'
    and o.order_delivered_customer_date is not null
)
select
    v.safra as safra,
    count(v.pedido) as qtd_pedidos,
    sum(v.atrasado) as qtd_atrasados,
    round(qtd_atrasados/qtd_pedidos, 2) as perc_atrasados
from olist.default.products p
inner join vendas v
    on p.product_id = v.produto
inner join olist.default.order_reviews orv
    on v.pedido = orv.order_id
where p.product_category_name = 'cama_mesa_banho'
group by v.safra
order by v.safra               
''').toPandas()

df['safra'] = pd.to_datetime(df['safra'])

plt.figure(figsize=(14, 6))
plt.plot(df['safra'], df['perc_atrasados'], label='Cama Mesa e Banho')
plt.title('Percentual de Pedidos Atrasados por Safra para Cama Mesa e Banho')
plt.xlabel('Safra')
plt.ylabel('Percentual de Pedidos Atrasados (%)')
plt.grid(axis='y')
plt.grid(axis='x')
plt.tight_layout()
plt.show()